In [43]:
import polars as pl
import pandas as pd

In [44]:
lf = pl.scan_csv('data/*.csv').unique()

In [45]:
lf.describe()

statistic,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
str,f64,str,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,str
"""count""",2.24540248e8,"""224540248""",2.24540248e8,2.24540248e8,2.24540248e8,2.24540248e8,2.24540248e8,"""224233051""","""140493135""","""198922593""",2.24457614e8,1.67524696e8,2.19333602e8,2.10119258e8,1.33935471e8,1.67664022e8,"""224540248"""
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0,"""307197""","""84047113""","""25617655""",82634.0,5.7015552e7,5.206646e6,1.442099e7,9.0604777e7,5.6876226e7,"""0"""
"""mean""",3.6811e8,null,34.440335,-94.969732,2.096187,217.17261,352.327606,null,null,null,45.929119,4.298639,59.502913,12.508982,4.014037,51.807152,null
"""std""",5.8429e7,null,7.672206,20.314681,6.374898,113.995473,179.60788,null,null,null,18.388324,5.501845,77.498178,11.006032,3.620624,20.799927,null
"""min""",0.0,"""2023-01-19T00:00:00""",-22.08184,-179.05592,0.0,0.0,0.0,"""############""","""IMO0000000""","""-""",0.0,0.0,0.0,0.0,0.0,0.0,"""A"""
"""25%""",3.6684328e8,null,29.55506,-117.22839,0.0,125.9,191.0,null,null,null,31.0,0.0,17.0,6.0,2.0,33.0,null
"""50%""",3.6746036e8,null,30.71743,-90.6055,0.0,236.6,511.0,null,null,null,37.0,0.0,25.0,8.0,3.0,52.0,null
"""75%""",3.6806394e8,null,40.06814,-80.46134,0.5,319.1,511.0,null,null,null,57.0,11.0,53.0,14.0,5.1,60.0,null
"""max""",1.0723e9,"""2024-01-22T23:59:59""",86.67657,178.5857,102.3,409.5,511.0,"""ZYANA K""","""IMO999999999""","""\""",255.0,15.0,894.0,126.0,28.2,255.0,"""B"""


In [46]:
df = (
    #keep only cargo ships (NOTE this may need to be expanded)
    lf.filter(pl.col('VesselType')==70)
    #keep cols of interest
    .select('MMSI', 'BaseDateTime','LAT', 'LON', 'SOG', 'COG', 'Heading',
            'VesselName', 'IMO')
    #give pythonic names
    .rename({
        'MMSI':'mmsi',
        'BaseDateTime':'time',
        'LAT':'lat',
        'LON':'lon',
        'SOG':'speed',
        'COG':'course',
        'Heading':'heading',
        'VesselName':'vessel_name',
        'IMO':'imo'
    })
    #clean cols
    .with_columns(
        #strip IMO prefix and cast to int
        imo = pl.col('imo').str.strip_prefix('IMO').cast(pl.Int64),
        #clean course and heading 
        course = pl.col('course').replace(360.0,None),
        heading = pl.col('heading').replace(511.0,None)
    )
    #cast
    .cast({
        #'mmsi':pl.Categorical,
        'time':pl.Datetime,
        'vessel_name':pl.Categorical,
        #'imo':pl.Categorical
    })
    #collect
    .collect()
)
df.describe()

/var/folders/dc/sgfd2dls28n4y4v4_jw03ly40000gp/T/ipykernel_98105/1769415228.py:35: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  .collect()


statistic,mmsi,time,lat,lon,speed,course,heading,vessel_name,imo
str,f64,str,f64,f64,f64,f64,f64,str,f64
"""count""",1.485288e7,"""14852880""",1.485288e7,1.485288e7,1.485288e7,1.4489186e7,1.3483563e7,"""14852880""",1.4316802e7
"""null_count""",0.0,"""0""",0.0,0.0,0.0,363694.0,1.369317e6,"""0""",536078.0
"""mean""",4.1057e8,"""2023-08-22 18:36:29.728394""",34.829989,-93.07851,4.799632,184.594675,184.427295,null,1.2472e7
"""std""",1.2133e8,null,8.134211,24.667199,6.567469,103.814063,105.769955,null,5.3808e7
"""min""",2.05087e8,"""2023-01-19 00:00:00""",5.47742,-179.05592,0.0,0.0,0.0,null,0.0
"""25%""",3.38196e8,"""2023-01-27 04:39:02""",29.29495,-118.168,0.0,98.6,93.0,null,9.271195e6
"""50%""",3.6769128e8,"""2024-01-04 01:23:44""",32.83382,-89.90983,0.1,186.8,181.0,null,9.516478e6
"""75%""",5.38005482e8,"""2024-01-11 20:38:19""",40.72742,-79.67974,10.5,272.0,278.0,null,9.710505e6
"""max""",7.50237e8,"""2024-01-22 23:59:59""",50.21838,147.34451,102.3,359.9,360.0,null,1.0000e9


In [47]:
df.head()

mmsi,time,lat,lon,speed,course,heading,vessel_name,imo
i64,datetime[μs],f64,f64,f64,f64,f64,cat,i64
538004675,2024-01-08 16:01:10,48.35749,-124.17153,8.2,292.6,289.0,"""MELIA""",9286968
229340000,2023-01-29 04:54:42,40.09908,-73.40117,9.9,137.2,139.0,"""CMA CGM MALTA""",9301433
366698000,2023-01-20 18:32:08,32.85027,-79.94021,0.0,0.0,55.0,"""CAPE DECISION""",7310911
367348000,2024-01-10 08:49:36,40.45479,-74.05381,0.0,205.2,28.0,"""USNS 1ST LT B. LOPEZ""",8302442
229671000,2023-01-21 04:05:26,18.03215,-63.09965,0.5,319.6,null,"""AQUAMARIN""",9551674


In [48]:
df.write_parquet('data/2024_01.parquet')

In [49]:
df2 = (
    df
    .with_columns(pl.col('time').dt.day().alias('day'))
    .group_by('day')
    .agg(
        pl.col('mmsi').n_unique()
    )
)
df2.describe()

statistic,day,mmsi
str,f64,f64
"""count""",31.0,31.0
"""null_count""",0.0,0.0
"""mean""",16.0,956.483871
"""std""",9.092121,41.857593
"""min""",1.0,877.0
"""25%""",9.0,931.0
"""50%""",16.0,947.0
"""75%""",24.0,995.0
"""max""",31.0,1038.0


In [50]:
df3 = (
    df.group_by('mmsi').agg(pl.col('time').count())
)
df3.describe()

statistic,mmsi,time
str,f64,f64
"""count""",3425.0,3425.0
"""null_count""",0.0,0.0
"""mean""",4.2029e8,4336.607299
"""std""",1.3031e8,4505.311034
"""min""",2.05087e8,1.0
"""25%""",3.16035905e8,998.0
"""50%""",3.71059e8,3363.0
"""75%""",5.38008317e8,6109.0
"""max""",7.50237e8,37249.0


In [ ]:
https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2024/AIS_2024_02_09.zip